In [1]:
import numpy as np 
import pandas as pd
import torch
from torch import nn 
from torch.utils.data import DataLoader,random_split,Dataset

In [2]:
import os

In [3]:
texts = [] 
labels = [] 
for i in range(1,9): 
    dirname = f'gbab{i}'
    for j in range(1,len(os.listdir(dirname))+1): 
        filename = dirname + '/'+f'hekayat{j}.txt'
        with open(filename,encoding='utf-8') as f : 
            texts.append(f.readlines()[0]);
            labels.append(i-1)

In [4]:
import re

In [5]:
texts = [re.sub(r'[\u064B-\u0652]','',t) for t in texts];
texts[0]

'پادشاهی را شنیدم به کشتن اسیری اشارت کرد بی چاره در آن حالت نومیدی ملک را دشنام دادن گرفت و سقط گفتن که گفته اند هر که دست از جان بشوید هر چه در دل دارد بگوید وقت ضرورت چو نماند گریز دست بگیرد سر شمشیر تیز إذا یئس الإنسان طال لسانه کسنور مغلوب یصول علی الکلب ملک پرسید چه می گوید یکی از وزرای نیک محضر گفت ای خداوند همی گوید و الکاظمین الغیظ و العافین عن الناس ملک را رحمت آمد و از سر خون او درگذشت وزیر دیگر که ضد او بود گفت ابنای جنس ما را نشاید در حضرت پادشاهان جز به راستی سخن گفتن این ملک را دشنام داد و ناسزا گفت ملک روی از این سخن در هم آورد و گفت آن دروغ وی پسندیده تر آمد مرا زین راست که تو گفتی که روی آن در مصلحتی بود و بنای این بر خبثی و خردمندان گفته اند دروغی مصلحت آمیز به که راستی فتنه انگیز هر که شاه آن کند که او گوید حیف باشد که جز نکو گوید بر طاق ایوان فریدون نبشته بود جهان ای برادر نماند به کس دل اندر جهان آفرین بند و بس مکن تکیه بر ملک دنیا و پشت که بسیار کس چون تو پرورد و کشت چو آهنگ رفتن کند جان پاک چه بر تخت مردن چه بر روی خاک '

In [6]:
with open('stopwords.txt',encoding='utf-8') as f : 
    stopwords = set(f.read().split('\n'))

In [7]:
sentences = [t.split() for t in texts]

In [8]:
from collections import Counter
def make_vocab(sentences, stopwords = set(), min_freq= 10): 
    cnt = Counter(word for sentence in sentences for word in sentence); 
    sentences = [[word for word in sentence if word not in stopwords and len(word)>2 and cnt[word]>=min_freq]
                for sentence in sentences] 
    vocab = set(); 
    word2idx = {}; 
    idx2word = {} 
    index = 0; 
    pairs = [] 
    for sentence in sentences : 
        for i, word in enumerate(sentence): 
            if word not in vocab: 
                word2idx[word] = index; 
                idx2word[index] = word; 
                vocab.add(word); 
                index+=1; 
            start = max(i-3,0); 
            end = min(len(sentence),i+4); 
            for j in range(start,end): 
                if (i==j): continue; 
                pairs.append((word,sentence[j]));
    return pairs,vocab,word2idx,idx2word

In [9]:
pairs,vocab,word2idx,idx2word = make_vocab(sentences,stopwords,min_freq=0)

In [10]:
class Text(Dataset): 
    def __init__(self, pairs): 
        super().__init__(); 
        self.pairs = pairs; 
    def __len__(self): return len(self.pairs) 
    def __getitem__(self, index): 
        return self.pairs[index][0],self.pairs[index][1]

In [11]:
dataset = Text(pairs);
dataset[0]

('پادشاهی', 'شنیدم')

In [12]:
def collate_fn(batch): 
    x,y = zip(*batch); 
    x = torch.tensor([word2idx[t] for t in x],dtype = torch.int64); 
    y = torch.tensor([word2idx[t] for t in y],dtype = torch.int64); 
    return x, y

In [38]:
from torch.nn.functional import logsigmoid

In [44]:
class Skip_gram(nn.Module): 
    def __init__(self, vocab_size,embedding_size): 
        super().__init__(); 
        self.in_embedding = nn.Embedding(vocab_size,embedding_size); 
        self.out_embedding = nn.Embedding(vocab_size,embedding_size); 
    def forward(self, in_ids,out_ids,neg_ids): 
        in_embed = self.in_embedding(in_ids); 
        out_embed = self.out_embedding(out_ids); 
        neg_embed = self.out_embedding(neg_ids); 
        pos_score = torch.sum(in_embed * out_embed,dim=1); 
        pos_score = logsigmoid(pos_score);
        neg_score = torch.bmm(neg_embed,in_embed.unsqueeze(2)).squeeze(2);
        neg_score = torch.sum(logsigmoid(-neg_score),dim = 1); 
        return -(neg_score+pos_score).mean();

In [45]:
def gen_neg_samples(vocab_size,pos_id,k = 5): 
    neg = []
    while (len(neg)!=k): 
        w = np.random.randint(low=0,high=vocab_size,size=1)[0]; 
        if (w!=pos_id):neg.append(w);
    return  neg

In [15]:
loader = DataLoader(dataset,batch_size=64,shuffle=True,collate_fn=collate_fn);

In [46]:
model = Skip_gram(vocab_size=len(vocab),embedding_size=100); 
model

Skip_gram(
  (in_embedding): Embedding(6105, 100)
  (out_embedding): Embedding(6105, 100)
)

In [49]:
optimizer = torch.optim.Adam(lr=2e-3,params=model.parameters())

In [50]:
b = 0; 
for _ in range(4): 
    model.train();
    for x, y in loader : 
        optimizer.zero_grad();
        negs = torch.tensor([gen_neg_samples(len(vocab),t.item()) for t in x],dtype =torch.int64)
        loss = model(x,y,negs); 
        loss.backward(); 
        optimizer.step(); 
        b+=1;
        if (b%100==0): print(loss.item())

2.1333727836608887
2.547220230102539
2.4917523860931396
2.3459975719451904
2.730955123901367
3.4252984523773193
1.882370948791504
1.8936411142349243
1.9091267585754395
2.283538818359375
2.0655405521392822
2.793238639831543
1.7312777042388916
1.3811575174331665
1.7332048416137695
1.5779465436935425
1.2438122034072876
1.9197956323623657
1.8917874097824097
1.4953420162200928
1.7498544454574585
1.066554307937622
1.8110307455062866
1.5388410091400146
1.286677360534668
1.0565507411956787
1.2403383255004883
1.2044761180877686
1.2308852672576904
1.3060466051101685
1.3607048988342285
1.7267804145812988
0.6818877458572388
1.186461329460144
0.7782601714134216
1.3580278158187866
0.5462521314620972
0.7103424072265625
0.5929203629493713
0.6478626132011414
1.4068515300750732
1.0919530391693115
0.8499455451965332
0.8690090775489807
1.1679506301879883
0.6179260015487671
0.6931474208831787
1.236621379852295
0.8806464076042175
0.8063183426856995
0.824002206325531
0.8305677175521851
0.6808286905288696
1.0

In [52]:
embedding = model.in_embedding.weight
def topk(word, k=5): 
    scores = embedding[word2idx[word]]@embedding.t() 
    ids = scores.argsort()[-k:].tolist(); 
    words = [idx2word[Id] for Id in ids]
    return words

In [57]:
topk('امیر')

['ثنایی', 'پوستینی', 'آزرم', 'برهاند', 'امیر']

In [59]:
cleaned_sentences = [[word for word in sentence if word in vocab]
                     for sentence in sentences]
cleaned_sentences[0]

['پادشاهی',
 'شنیدم',
 'کشتن',
 'اسیری',
 'اشارت',
 'چاره',
 'حالت',
 'نومیدی',
 'ملک',
 'دشنام',
 'سقط',
 'دست',
 'جان',
 'بشوید',
 'وقت',
 'ضرورت',
 'نماند',
 'گریز',
 'دست',
 'شمشیر',
 'تیز',
 'إذا',
 'یئس',
 'الإنسان',
 'طال',
 'لسانه',
 'کسنور',
 'مغلوب',
 'یصول',
 'علی',
 'الکلب',
 'ملک',
 'پرسید',
 'وزرای',
 'نیک',
 'محضر',
 'خداوند',
 'همی',
 'الکاظمین',
 'الغیظ',
 'العافین',
 'الناس',
 'ملک',
 'رحمت',
 'خون',
 'درگذشت',
 'وزیر',
 'ابنای',
 'جنس',
 'نشاید',
 'حضرت',
 'پادشاهان',
 'سخن',
 'ملک',
 'دشنام',
 'ناسزا',
 'ملک',
 'سخن',
 'دروغ',
 'پسندیده',
 'زین',
 'مصلحتی',
 'بنای',
 'خبثی',
 'خردمندان',
 'دروغی',
 'مصلحت',
 'آمیز',
 'فتنه',
 'انگیز',
 'شاه',
 'حیف',
 'نکو',
 'طاق',
 'ایوان',
 'فریدون',
 'نبشته',
 'جهان',
 'برادر',
 'نماند',
 'اندر',
 'جهان',
 'آفرین',
 'بند',
 'مکن',
 'تکیه',
 'ملک',
 'دنیا',
 'پرورد',
 'کشت',
 'آهنگ',
 'رفتن',
 'جان',
 'پاک',
 'تخت',
 'مردن',
 'خاک']

In [60]:
X = [] 
for sentence in cleaned_sentences: 
    if len(sentence)==0: 
        X.append(np.zeros(100)); 
        continue; 
    l = np.array([embedding[word2idx[t]].detach().numpy() for t in sentence]).mean(0); 
    X.append(l);
X = np.array(X)

In [66]:
from sklearn.model_selection import train_test_split
labels = np.array(labels)
x_train,x_test,y_train,y_test = train_test_split(X,labels,test_size=0.2,shuffle=True)

In [68]:
from lightgbm import LGBMClassifier

In [69]:
clf = LGBMClassifier(verbose=-1,n_estimators=4000); 
clf.fit(x_train,y_train); 
predict = clf.predict(x_test)

C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\msadr\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 1036

In [70]:
predict

array([1, 2, 0, 0, 0, 0, 7, 1, 7, 7, 0, 1, 0, 7, 1, 1, 0, 7, 7, 7, 7, 7,
       7, 4, 0, 7, 7, 7, 1, 7, 1, 7, 7, 0, 0, 7, 7, 7, 6, 0, 0, 7, 7, 0,
       7, 1, 0, 1, 0, 7, 1, 6, 0, 7, 1, 1, 7, 7])

In [72]:
from sklearn.metrics import accuracy_score,f1_score

In [73]:
print(accuracy_score(predict,y_test))

0.27586206896551724


In [74]:
print(f1_score(predict,y_test,average='macro'))

0.1467391304347826


In [75]:
1/8

0.125